In [2]:
!pip install scipy folium scikit-learn osmnx networkx shapely

import os
from pathlib import Path
import pandas as pd
from typing import Dict, List, Optional, Tuple
import numpy as np
from datetime import datetime
import osmnx as ox
import networkx as nx
import scipy
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 8.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [12]:
# パスの設定
current_dir = Path('/app')  # Dockerコンテナ内のワークディレクトリを直接指定
target_path = current_dir / 'data' / 'raw' / 'DataInput' / '202311'

print(f"検索対象のパス: {target_path}")

# 各テーブル用のDataFrame
df_vehicles = None
df_bus_loc = None
df_passengers = None
df_agency = None
df_routes = None
df_trips = None
df_calendar = None
df_stops = None

# フォルダ内のCSVファイルを処理
if not target_path.exists():
    print(f"指定されたパスが見つかりません: {target_path}")
else:
    # CSVファイルのみを取得
    csv_files = [f for f in target_path.glob('*.csv')]
    
    if not csv_files:
        print("CSVファイルが見つかりません")
    
    # 各CSVファイルを読み込んで対応するDataFrameに格納
    for csv_file in csv_files:
        file_name = csv_file.name.lower()  # ファイル名を小文字に変換
        print(f"処理中: {csv_file.name}")
        
        try:
            if 'vehicles' in file_name:
                df_vehicles = pd.read_csv(csv_file)
                print("vehicles データを読み込みました")
            elif 'bus_loc' in file_name:
                df_bus_loc = pd.read_csv(csv_file)
                print("bus_loc データを読み込みました")
            elif 'passengers' in file_name:
                df_passengers = pd.read_csv(csv_file)
                print("passengers データを読み込みました")
            elif 'agency' in file_name:
                df_agency = pd.read_csv(csv_file)
                print("agency データを読み込みました")
            elif 'routes' in file_name:
                df_routes = pd.read_csv(csv_file)
                print("routes データを読み込みました")
            elif 'trips' in file_name:
                df_trips = pd.read_csv(csv_file)
                print("trips データを読み込みました")
            elif 'calendar' in file_name:
                df_calendar = pd.read_csv(csv_file)
                print("calendar データを読み込みました")
            elif 'stops' in file_name:
                df_stops = pd.read_csv(csv_file)
                print("stops データを読み込みました")
            else:
                print(f"未知のファイル形式です: {csv_file.name}")
        except Exception as e:
            print(f"エラーが発生しました - {csv_file.name}: {e}")

# 読み込んだデータの確認
for df_name, df in [
    ('vehicles', df_vehicles),
    ('bus_loc', df_bus_loc),
    ('passengers', df_passengers),
    ('agency', df_agency),
    ('routes', df_routes),
    ('trips', df_trips),
    ('calendar', df_calendar),
    ('stops', df_stops)
]:
    if df is not None:
        print(f"\n{df_name} データの形状: {df.shape}")
        display(df.head(3))
    else:
        print(f"\n{df_name} データは読み込まれていません")

検索対象のパス: /app/data/raw/DataInput/202311
処理中: agency.csv
agency データを読み込みました
処理中: bus_loc.csv
bus_loc データを読み込みました
処理中: calendar.csv
calendar データを読み込みました
処理中: passengers.csv
passengers データを読み込みました
処理中: routes.csv
routes データを読み込みました
処理中: stops.csv
stops データを読み込みました
処理中: trips.csv
trips データを読み込みました

vehicles データは読み込まれていません

bus_loc データの形状: (2164770, 9)


,Unnamed: 0,event_id,trip_id,from_stop_id,to_stop_id,date,time,bus_lat,bus_lon
0,0,T0108000001_2023-11-01,T0108000001,S010800006700100,S010800006800100,2023-11-01,07:00:00,42.990305,141.358102
1,1,T0108000001_2023-11-01,T0108000001,S010800006800100,S010800006900100,2023-11-01,07:00:00,42.989159,141.353762
2,2,T0108000001_2023-11-01,T0108000001,S010800006900100,S010800007000100,2023-11-01,07:01:00,42.985279,141.347964



passengers データの形状: (151243, 7)


,route_id,from_stop_id,to_stop_id,from_stop_name,to_stop_name,date,total_passengers
0,R010800823,S010800000600100,S010800001000100,すすきの,北海学園大工学部前,20231101,5
1,R010800822,S010800000600100,S010800001000100,すすきの,北海学園大工学部前,20231101,3
2,R010800824,S010800000600100,S010800001000100,すすきの,北海学園大工学部前,20231101,5



agency データの形状: (1, 8)


...

,trip_id,route_id,service_id,trip_headsign,trip_short_name,direction_id,shape_id,wheelchair_accessible,bikes_allowed
0,T0108000001,R010800818,S000001,定山渓車庫前,NaN,0,NaN,0,0
1,T0108000002,R010800818,S000001,定山渓車庫前,NaN,0,NaN,0,0
2,T0108000003,R010800818,S000001,定山渓車庫前,NaN,0,NaN,0,0



calendar データの形状: (8, 10)


,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,S000001,1,1,1,1,1,0,0,20231101,20240430
1,S000002,0,0,0,0,0,1,1,20231103,20240429
2,S000003,1,1,1,1,1,1,1,20231101,20240430



stops データの形状: (394, 11)


,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding
0,S010800000100100,NaN,札幌駅,43.066233,141.353966,Z000570,NaN,0,NaN,NaN,NaN
1,S010800000100200,NaN,札幌駅,43.066441,141.353931,Z000570,NaN,0,NaN,NaN,NaN
2,S010800000100300,NaN,札幌駅,43.066702,141.353862,Z000570,NaN,0,NaN,NaN,NaN


In [14]:
# ... existing imports ...

def create_base_geojson(routes_df: pd.DataFrame, stops_df: pd.DataFrame) -> Dict:
    def generate_random_color() -> str:
        return '#{:06x}'.format(np.random.randint(0, 0xFFFFFF))
    
    # 処理対象の年月を設定
    target_months = ["202311"]  # 後で動的に生成する場合は拡張可能
    
    features = []
    
    for _, route in routes_df.iterrows():
        # この路線に関連する停留所を取得（仮の実装）
        route_stops = stops_df.head(3)
        
        stops_info = []
        coordinates = []
        
        for _, stop in route_stops.iterrows():
            # 各月のメトリクスを作成
            monthly_metrics = {
                month: {
                    "passengers_on": 0,
                    "passengers_off": 0,
                    "revenue": 0,
                    "cost": 0,
                    "average_passenger_density": 0,
                    "emissions": 0  # 始発停留所は0、それ以外は前の停留所からの排出量
                }
                for month in target_months
            }
            
            stop_info = {
                "stop_id": stop['stop_id'],
                "stop_name": stop['stop_name'],
                "coordinates": [float(stop['stop_lon']), float(stop['stop_lat'])],
                "metrics": monthly_metrics
            }
            stops_info.append(stop_info)
            coordinates.append([float(stop['stop_lon']), float(stop['stop_lat'])])
        
        # 路線全体のメトリクスを作成
        route_metrics = {
            month: {
                "emissions": 0,
                "od_distance": {
                    "car": 0,
                    "motorbike": 0,
                    "train": 0,
                    "non_emission": 0
                },
                "environmental_value": {
                    "car": 0,
                    "motorbike": 0,
                    "train": 0,
                    "non_emission": 0
                }
            }
            for month in target_months
        }
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {
                "route_id": route['route_id'],
                "route_name": route['route_long_name'],
                "color": generate_random_color(),
                "metrics": route_metrics,
                "stops": stops_info
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# GeoJSONの作成と保存
geojson_data = create_base_geojson(df_routes, df_stops)
interim_path = interim_dir / 'base_routes.geojson'
with open(interim_path, 'w', encoding='utf-8') as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=2)

print(f"基本GeoJSONを保存しました: {interim_path}")

基本GeoJSONを保存しました: /app/data/interim/base_routes.geojson


In [18]:
def calculate_route_metrics(
    route_id: str,
    passengers_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    vehicles_df: pd.DataFrame,
    trips_df: pd.DataFrame,  # trips_dfを引数に追加
    target_month: str
) -> dict:
    """路線全体のメトリクスを計算する"""
    
    # 該当路線の乗客データを抽出
    route_passengers = passengers_df[passengers_df['route_id'] == route_id]
    
    # 該当路線のバス位置データを抽出
    route_bus_loc = bus_loc_df[bus_loc_df['trip_id'].isin(
        trips_df[trips_df['route_id'] == route_id]['trip_id']
    )]
    
    # CO2排出量の計算（仮実装）
    emissions = 0
    if not route_bus_loc.empty and 'vehicle_id' in route_bus_loc.columns and vehicles_df is not None:
        try:
            vehicle_ids = route_bus_loc['vehicle_id'].unique()
            vehicles_info = vehicles_df[vehicles_df['vehicle_id'].isin(vehicle_ids)]
            # 燃費情報を使用してCO2排出量を計算
            # ... 詳細な計算ロジックを実装 ...
        except Exception as e:
            print(f"車両データの処理中にエラーが発生しました: {e}")
            # エラーが発生しても処理を継続
    
    # OD距離の計算（仮実装）
    total_passengers = route_passengers['total_passengers'].sum() if not route_passengers.empty else 0
    od_distance = {
        "car": total_passengers * 0.3,  # 仮の係数
        "motorbike": total_passengers * 0.1,
        "train": total_passengers * 0.4,
        "non_emission": total_passengers * 0.2
    }
    
    # 環境価値の計算（仮実装）
    environmental_value = {
        "car": od_distance["car"] * 50,  # 仮の係数
        "motorbike": od_distance["motorbike"] * 30,
        "train": od_distance["train"] * 20,
        "non_emission": 0
    }
    
    return {
        "emissions": float(emissions),  # 確実に数値型に変換
        "od_distance": {k: float(v) for k, v in od_distance.items()},
        "environmental_value": {k: float(v) for k, v in environmental_value.items()}
    }

def calculate_stop_metrics(
    stop_id: str,
    route_id: str,
    passengers_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    target_month: str
) -> dict:
    """停留所ごとのメトリクスを計算する"""
    
    # 乗降客数の計算
    stop_passengers = passengers_df[
        (passengers_df['from_stop_id'] == stop_id) | 
        (passengers_df['to_stop_id'] == stop_id)
    ]
    
    passengers_on = stop_passengers[
        stop_passengers['from_stop_id'] == stop_id
    ]['total_passengers'].sum()
    
    passengers_off = stop_passengers[
        stop_passengers['to_stop_id'] == stop_id
    ]['total_passengers'].sum()
    
    # 運賃収入の計算（仮実装）
    revenue = passengers_on * 200  # 仮の運賃単価
    
    # コストの計算（仮実装）
    cost = passengers_on * 150  # 仮の運営コスト
    
    # 平均乗車密度の計算（仮実装）
    avg_density = passengers_on / 100  # 仮の係数
    
    # CO2排出量の計算（仮実装）
    emissions = passengers_on * 0.5  # 仮の係数
    
    return {
        "passengers_on": int(passengers_on),
        "passengers_off": int(passengers_off),
        "revenue": float(revenue),
        "cost": float(cost),
        "average_passenger_density": float(avg_density),
        "emissions": float(emissions)
    }

def get_route_stops(
    route_id: str, 
    trips_df: pd.DataFrame, 
    bus_loc_df: pd.DataFrame,
    stops_df: pd.DataFrame  # stops_dfを引数に追加
) -> pd.DataFrame:
    """路線に属する停留所を取得する"""
    # 路線のtrip_idを取得
    route_trips = trips_df[trips_df['route_id'] == route_id]['trip_id']
    
    # tripに関連する停留所を取得
    route_stops = bus_loc_df[
        bus_loc_df['trip_id'].isin(route_trips)
    ][['from_stop_id', 'to_stop_id']].drop_duplicates()
    
    # 停留所IDのリストを作成（from_stop_idとto_stop_idを統合）
    stop_ids = pd.concat([
        route_stops['from_stop_id'],
        route_stops['to_stop_id']
    ]).unique()
    
    return stops_df[stops_df['stop_id'].isin(stop_ids)]

def create_detailed_geojson(
    routes_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    trips_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    passengers_df: pd.DataFrame,
    vehicles_df: pd.DataFrame
) -> Dict:
    """より詳細なGeoJSONを作成する"""
    
    target_months = ["202311"]  # 後で動的に生成する場合は拡張可能
    features = []
    
    for _, route in routes_df.iterrows():
        route_id = route['route_id']
        
        # 路線に属する停留所を取得
        route_stops = get_route_stops(route_id, trips_df, bus_loc_df, stops_df)
        
        stops_info = []
        coordinates = []
        
        for _, stop in route_stops.iterrows():
            # 各月のメトリクスを計算
            monthly_metrics = {
                month: calculate_stop_metrics(
                    stop['stop_id'],
                    route_id,
                    passengers_df,
                    bus_loc_df,
                    month
                )
                for month in target_months
            }
            
            stop_info = {
                "stop_id": stop['stop_id'],
                "stop_name": stop['stop_name'],
                "coordinates": [float(stop['stop_lon']), float(stop['stop_lat'])],
                "metrics": monthly_metrics
            }
            stops_info.append(stop_info)
            coordinates.append([float(stop['stop_lon']), float(stop['stop_lat'])])
        
        # 路線全体のメトリクスを計算
        route_metrics = {
            month: calculate_route_metrics(
                route_id,
                passengers_df,
                bus_loc_df,
                vehicles_df,
                trips_df,
                month
            )
            for month in target_months
        }
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {
                "route_id": route_id,
                "route_name": route['route_long_name'],
                "color": f"#{hash(route_id) % 0xFFFFFF:06x}",  # route_idに基づいた一貫した色
                "metrics": route_metrics,
                "stops": stops_info
            }
        }
        features.append(feature)
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# 詳細なGeoJSONの作成と保存
detailed_geojson = create_detailed_geojson(
    df_routes,
    df_stops,
    df_trips,
    df_bus_loc,
    df_passengers,
    df_vehicles
)

# 中間データとして保存
detailed_path = interim_dir / 'detailed_routes.geojson'
with open(detailed_path, 'w', encoding='utf-8') as f:
    json.dump(detailed_geojson, f, ensure_ascii=False, indent=2)

print(f"詳細なGeoJSONを保存しました: {detailed_path}")

詳細なGeoJSONを保存しました: /app/data/interim/detailed_routes.geojson


In [19]:
import numpy as np
from typing import List, Tuple

def optimize_route_coordinates(
    stops: List[dict],
    route_id: str
) -> List[List[float]]:
    """
    停留所間の経路を最適化する
    現在は直線補間だが、後でOpenStreetMapなどの道路データを使用して実際の経路に沿うように改善可能
    """
    optimized_coords = []
    
    for i in range(len(stops) - 1):
        current_stop = stops[i]
        next_stop = stops[i + 1]
        
        # 現在の停留所の座標を追加
        current_coords = current_stop["coordinates"]
        optimized_coords.append(current_coords)
        
        # 停留所間の中間点を補完（現在は単純な直線補間）
        num_points = 5  # 補完点の数
        for j in range(1, num_points):
            ratio = j / num_points
            interpolated_lon = current_coords[0] + (next_stop["coordinates"][0] - current_coords[0]) * ratio
            interpolated_lat = current_coords[1] + (next_stop["coordinates"][1] - current_coords[1]) * ratio
            optimized_coords.append([interpolated_lon, interpolated_lat])
    
    # 最後の停留所の座標を追加
    optimized_coords.append(stops[-1]["coordinates"])
    
    return optimized_coords

def validate_coordinates(coordinates: List[List[float]]) -> bool:
    """座標データのバリデーション"""
    if not coordinates:
        return False
    
    for coord in coordinates:
        if len(coord) != 2:
            return False
        lon, lat = coord
        # 緯度・経度の基本的な範囲チェック
        if not (-180 <= lon <= 180) or not (-90 <= lat <= 90):
            return False
    return True

def create_optimized_geojson(
    routes_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    trips_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    passengers_df: pd.DataFrame,
    vehicles_df: pd.DataFrame
) -> Dict:
    """最適化されたGeoJSONを作成する"""
    
    target_months = ["202311"]
    features = []
    
    for _, route in routes_df.iterrows():
        route_id = route['route_id']
        print(f"路線 {route_id} の処理中...")
        
        # 路線に属する停留所を取得
        route_stops = get_route_stops(route_id, trips_df, bus_loc_df, stops_df)
        
        if route_stops.empty:
            print(f"警告: 路線 {route_id} に有効な停留所がありません")
            continue
        
        stops_info = []
        
        # 停留所情報の作成
        for _, stop in route_stops.iterrows():
            try:
                monthly_metrics = {
                    month: calculate_stop_metrics(
                        stop['stop_id'],
                        route_id,
                        passengers_df,
                        bus_loc_df,
                        month
                    )
                    for month in target_months
                }
                
                stop_info = {
                    "stop_id": stop['stop_id'],
                    "stop_name": stop['stop_name'],
                    "coordinates": [float(stop['stop_lon']), float(stop['stop_lat'])],
                    "metrics": monthly_metrics
                }
                stops_info.append(stop_info)
            except Exception as e:
                print(f"警告: 停留所 {stop['stop_id']} の処理中にエラーが発生しました: {e}")
                continue
        
        if not stops_info:
            print(f"警告: 路線 {route_id} の有効な停留所情報がありません")
            continue
        
        # 座標の最適化
        try:
            coordinates = optimize_route_coordinates(stops_info, route_id)
            
            if not validate_coordinates(coordinates):
                print(f"警告: 路線 {route_id} の座標データが無効です")
                continue
                
        except Exception as e:
            print(f"警告: 路線 {route_id} の座標最適化中にエラーが発生しました: {e}")
            continue
        
        # 路線全体のメトリクス計算
        try:
            route_metrics = {
                month: calculate_route_metrics(
                    route_id,
                    passengers_df,
                    bus_loc_df,
                    vehicles_df,
                    trips_df,
                    month
                )
                for month in target_months
            }
        except Exception as e:
            print(f"警告: 路線 {route_id} のメトリクス計算中にエラーが発生しました: {e}")
            continue
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {
                "route_id": route_id,
                "route_name": route['route_long_name'],
                "color": f"#{hash(route_id) % 0xFFFFFF:06x}",
                "metrics": route_metrics,
                "stops": stops_info
            }
        }
        features.append(feature)
        print(f"路線 {route_id} の処理が完了しました")
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# 最適化されたGeoJSONの作成と保存
optimized_geojson = create_optimized_geojson(
    df_routes,
    df_stops,
    df_trips,
    df_bus_loc,
    df_passengers,
    df_vehicles
)

# 中間データとして保存
optimized_path = interim_dir / 'optimized_routes.geojson'
with open(optimized_path, 'w', encoding='utf-8') as f:
    json.dump(optimized_geojson, f, ensure_ascii=False, indent=2)

print(f"最適化されたGeoJSONを保存しました: {optimized_path}")

路線 R010800818 の処理中...
路線 R010800818 の処理が完了しました
路線 R010800819 の処理中...
路線 R010800819 の処理が完了しました
路線 R010800005 の処理中...
路線 R010800005 の処理が完了しました
路線 R010800007 の処理中...
路線 R010800007 の処理が完了しました
路線 R010800820 の処理中...
路線 R010800820 の処理が完了しました
路線 R010800002 の処理中...
路線 R010800002 の処理が完了しました
路線 R010800003 の処理中...
路線 R010800003 の処理が完了しました
路線 R010800821 の処理中...
路線 R010800821 の処理が完了しました
路線 R010800822 の処理中...
路線 R010800822 の処理が完了しました
路線 R010800033 の処理中...
路線 R010800033 の処理が完了しました
路線 R010800823 の処理中...
路線 R010800823 の処理が完了しました
路線 R010800824 の処理中...
路線 R010800824 の処理が完了しました
路線 R010800825 の処理中...
路線 R010800825 の処理が完了しました
路線 R010800826 の処理中...
路線 R010800826 の処理が完了しました
路線 R010800009 の処理中...
路線 R010800009 の処理が完了しました
路線 R010800011 の処理中...
路線 R010800011 の処理が完了しました
路線 R010800013 の処理中...
路線 R010800013 の処理が完了しました
路線 R010800014 の処理中...
路線 R010800014 の処理が完了しました
路線 R010800016 の処理中...
路線 R010800016 の処理が完了しました
路線 R010800018 の処理中...
路線 R010800018 の処理が完了しました
路線 R010800020 の処理中...
路線 R010800020 の処理が完了しました
路線 R010800021

In [41]:
def calculate_emissions(
    route_bus_loc: pd.DataFrame,
    vehicles_df: Optional[pd.DataFrame],
    distance: float
) -> float:
    """CO2排出量をより正確に計算する"""
    if vehicles_df is None or vehicles_df.empty:
        # 車両データがない場合は距離当たりの標準的な排出量で計算
        return distance * 0.5  # 仮の係数: 0.5kg/km
    
    total_emissions = 0
    try:
        for _, vehicle in vehicles_df.iterrows():
            # 燃費効率に基づいてCO2排出量を計算
            fuel_efficiency = vehicle.get('fuel_efficiency', 10)  # デフォルト: 10km/L
            fuel_type = vehicle.get('fuel_type', 'diesel')
            
            # 燃料タイプごとのCO2排出係数 (kg/L)
            emission_factors = {
                'diesel': 2.58,
                'gasoline': 2.32,
                'cng': 2.02,
                'electric': 0
            }
            
            # 燃料消費量とCO2排出量の計算
            fuel_consumption = distance / fuel_efficiency
            emissions = fuel_consumption * emission_factors.get(fuel_type, 2.58)
            total_emissions += emissions
            
    except Exception as e:
        print(f"排出量計算中にエラーが発生: {e}")
        return distance * 0.5  # エラー時は標準値を使用
    
    return total_emissions

def calculate_revenue_and_cost(
    passengers: int,
    distance: float,
    stop_count: int
) -> Tuple[float, float]:
    """運賃収入とコストをより現実的に計算する"""
    # 基本運賃（距離に応じて変動）
    base_fare = 200  # 基本運賃
    distance_fare = distance * 10  # 距離あたりの追加運賃
    average_fare = base_fare + distance_fare
    
    # 収入の計算（割引なども考慮）
    revenue = passengers * average_fare * 0.9  # 10%の割引を考慮
    
    # コストの計算
    fuel_cost = distance * 100  # 燃料費
    maintenance_cost = distance * 50  # メンテナンス費
    labor_cost = stop_count * 1000  # 人件費
    
    total_cost = fuel_cost + maintenance_cost + labor_cost
    
    return revenue, total_cost

def calculate_environmental_value(
    od_distance: Dict[str, float],
    emissions: float
) -> Dict[str, float]:
    """環境価値をより詳細に計算する"""
    # CO2排出削減価値（円/kg）
    co2_value = 10000  # 仮の係数
    
    # 交通手段ごとの環境負荷係数
    transport_factors = {
        'car': 0.2,
        'motorbike': 0.1,
        'train': 0.05,
        'non_emission': 0
    }
    
    # 交通手段ごとの環境価値を計算
    environmental_values = {}
    for mode, distance in od_distance.items():
        # CO2削減量の計算
        reduced_emissions = distance * transport_factors[mode]
        # 環境価値の計算
        env_value = reduced_emissions * co2_value
        environmental_values[mode] = env_value
    
    return environmental_values

def calculate_passenger_density(
    passengers: int,
    vehicle_capacity: int = 50  # デフォルトの車両定員
) -> float:
    """平均乗車密度を計算する"""
    return min(1.0, passengers / vehicle_capacity)

def calculate_detailed_stop_metrics(
    stop_id: str,
    route_id: str,
    passengers_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    vehicles_df: Optional[pd.DataFrame],
    target_month: str
) -> dict:
    """停留所ごとのメトリクスを詳細に計算する"""
    
    # 乗降客数の計算
    stop_passengers = passengers_df[
        (passengers_df['from_stop_id'] == stop_id) | 
        (passengers_df['to_stop_id'] == stop_id)
    ]
    
    passengers_on = stop_passengers[
        stop_passengers['from_stop_id'] == stop_id
    ]['total_passengers'].sum()
    
    passengers_off = stop_passengers[
        stop_passengers['to_stop_id'] == stop_id
    ]['total_passengers'].sum()
    
    # 停留所間の距離を計算（仮実装）
    distance = 1.0  # km
    
    # 収入とコストの計算
    revenue, cost = calculate_revenue_and_cost(
        int(passengers_on),
        distance,
        1  # この停留所のみ
    )
    
    # 平均乗車密度の計算
    avg_density = calculate_passenger_density(int(passengers_on))
    
    # CO2排出量の計算
    emissions = calculate_emissions(
        bus_loc_df,
        vehicles_df,
        distance
    )
    
    return {
        "passengers_on": int(passengers_on),
        "passengers_off": int(passengers_off),
        "revenue": float(revenue),
        "cost": float(cost),
        "average_passenger_density": float(avg_density),
        "emissions": float(emissions)
    }

def calculate_detailed_route_metrics(
    route_id: str,
    passengers_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    vehicles_df: Optional[pd.DataFrame],
    trips_df: pd.DataFrame,
    target_month: str
) -> dict:
    """路線全体のメトリクスを詳細に計算する"""
    
    # 該当路線の乗客データを抽出
    route_passengers = passengers_df[passengers_df['route_id'] == route_id]
    total_passengers = route_passengers['total_passengers'].sum() if not route_passengers.empty else 0
    
    # 路線の総距離を計算（仮実装）
    total_distance = 10.0  # km
    
    # CO2排出量の計算
    emissions = calculate_emissions(
        bus_loc_df,
        vehicles_df,
        total_distance
    )
    
    # OD距離の計算（より現実的な比率で）
    od_distance = {
        "car": total_passengers * 0.4,      # 40%が車からの転換
        "motorbike": total_passengers * 0.1, # 10%がバイクからの転換
        "train": total_passengers * 0.3,     # 30%が電車からの転換
        "non_emission": total_passengers * 0.2  # 20%が徒歩・自転車からの転換
    }
    
    # 環境価値の計算
    environmental_value = calculate_environmental_value(od_distance, emissions)
    
    return {
        "emissions": float(emissions),
        "od_distance": {k: float(v) for k, v in od_distance.items()},
        "environmental_value": {k: float(v) for k, v in environmental_value.items()}
    }

# 既存のcreate_optimized_geojson関数を更新
def create_optimized_geojson(
    routes_df: pd.DataFrame,
    stops_df: pd.DataFrame,
    trips_df: pd.DataFrame,
    bus_loc_df: pd.DataFrame,
    passengers_df: pd.DataFrame,
    vehicles_df: Optional[pd.DataFrame]
) -> Dict:
    """最適化されたGeoJSONを作成する（詳細なメトリクス計算を使用）"""
    
    target_months = ["202311"]
    features = []
    
    for _, route in routes_df.iterrows():
        route_id = route['route_id']
        print(f"路線 {route_id} の処理中...")
        
        # 路線に属する停留所を取得
        route_stops = get_route_stops(route_id, trips_df, bus_loc_df, stops_df)
        
        if route_stops.empty:
            print(f"警告: 路線 {route_id} に有効な停留所がありません")
            continue
        
        stops_info = []
        
        # 停留所情報の作成
        for _, stop in route_stops.iterrows():
            try:
                monthly_metrics = {
                    month: calculate_detailed_stop_metrics(
                        stop['stop_id'],
                        route_id,
                        passengers_df,
                        bus_loc_df,
                        vehicles_df,
                        month
                    )
                    for month in target_months
                }
                
                stop_info = {
                    "stop_id": stop['stop_id'],
                    "stop_name": stop['stop_name'],
                    "coordinates": [float(stop['stop_lon']), float(stop['stop_lat'])],
                    "metrics": monthly_metrics
                }
                stops_info.append(stop_info)
            except Exception as e:
                print(f"警告: 停留所 {stop['stop_id']} の処理中にエラーが発生しました: {e}")
                continue
        
        if not stops_info:
            print(f"警告: 路線 {route_id} の有効な停留所情報がありません")
            continue
        
        # 座標の最適化
        try:
            coordinates = optimize_route_coordinates(stops_info, route_id)
            
            if not validate_coordinates(coordinates):
                print(f"警告: 路線 {route_id} の座標データが無効です")
                continue
                
        except Exception as e:
            print(f"警告: 路線 {route_id} の座標最適化中にエラーが発生しました: {e}")
            continue
        
        # 路線全体のメトリクス計算
        try:
            route_metrics = {
                month: calculate_detailed_route_metrics(
                    route_id,
                    passengers_df,
                    bus_loc_df,
                    vehicles_df,
                    trips_df,
                    month
                )
                for month in target_months
            }
        except Exception as e:
            print(f"警告: 路線 {route_id} のメトリクス計算中にエラーが発生しました: {e}")
            continue
        
        feature = {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordinates": coordinates
            },
            "properties": {
                "route_id": route_id,
                "route_name": route['route_long_name'],
                "color": f"#{hash(route_id) % 0xFFFFFF:06x}",
                "metrics": route_metrics,
                "stops": stops_info
            }
        }
        features.append(feature)
        print(f"路線 {route_id} の処理が完了しました")
    
    return {
        "type": "FeatureCollection",
        "features": features
    }

# 最適化されたGeoJSONの作成と保存
optimized_geojson = create_optimized_geojson(
    df_routes,
    df_stops,
    df_trips,
    df_bus_loc,
    df_passengers,
    df_vehicles
)

# 中間データとして保存
optimized_path = interim_dir / 'optimized_routes_detailed.geojson'
with open(optimized_path, 'w', encoding='utf-8') as f:
    json.dump(optimized_geojson, f, ensure_ascii=False, indent=2)

print(f"詳細な最適化GeoJSONを保存しました: {optimized_path}")


路線 R010800818 の処理中...


/usr/local/lib/python3.9/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


OpenStreetMapからの経路取得中にエラーが発生: scipy must be installed to search a projected graph


/usr/local/lib/python3.9/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


OpenStreetMapからの経路取得中にエラーが発生: scipy must be installed to search a projected graph


...

OpenStreetMapからの経路取得中にエラーが発生: scipy must be installed to search a projected graph


/usr/local/lib/python3.9/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


OpenStreetMapからの経路取得中にエラーが発生: scipy must be installed to search a projected graph


/usr/local/lib/python3.9/site-packages/osmnx/graph.py:191: FutureWarning: The expected order of coordinates in `bbox` will change in the v2.0.0 release to `(left, bottom, right, top)`.
  G = graph_from_bbox(


KeyboardInterrupt: 

In [37]:
import numpy as np
from typing import List, Dict, Tuple
import osmnx as ox
import networkx as nx
from math import radians, sin, cos, sqrt, atan2

def get_route_from_osm(
    start_coords: List[float],
    end_coords: List[float],
    mode: str = 'drive'
) -> List[List[float]]:
    """OpenStreetMapを使用して2点間の最適経路を取得する"""
    try:
        # 座標系を指定してグラフを取得（警告に対応したパラメータ設定）
        graph = ox.graph_from_point(
            center_point=(start_coords[1], start_coords[0]),
            dist=1000,
            network_type=mode,
            simplify=True,
            retain_all=True,  # clean_peripheryの代わりに使用
            truncate_by_edge=True
        )
        
        # グラフを投影
        graph_proj = ox.project_graph(graph)
        
        # 最寄りのノードを取得（投影されたグラフを使用）
        start_node = ox.distance.nearest_nodes(
            graph_proj,
            X=start_coords[0],
            Y=start_coords[1]
        )
        end_node = ox.distance.nearest_nodes(
            graph_proj,
            X=end_coords[0],
            Y=end_coords[1]
        )
        
        try:
            # 最短経路を計算
            route = nx.shortest_path(
                graph_proj,
                start_node,
                end_node,
                weight='length'
            )
            
            # 経路の座標リストを取得
            route_coords = []
            for node in route:
                coords = graph_proj.nodes[node]
                route_coords.append([coords['x'], coords['y']])
            
            return route_coords
            
        except nx.NetworkXNoPath:
            print(f"経路が見つかりませんでした。直線補間を使用します。")
            return [start_coords, end_coords]
            
    except Exception as e:
        print(f"OpenStreetMapからの経路取得中にエラーが発生: {e}")
        # エラー時は直線補間を返す
        return [start_coords, end_coords]

def optimize_route_coordinates(
    stops: List[dict],
    route_id: str
) -> List[List[float]]:
    """停留所間の経路を最適化する"""
    optimized_coords = []
    
    for i in range(len(stops) - 1):
        current_stop = stops[i]
        next_stop = stops[i + 1]
        
        try:
            # OpenStreetMapを使用して経路を取得
            segment_coords = get_route_from_osm(
                current_stop["coordinates"],
                next_stop["coordinates"]
            )
            optimized_coords.extend(segment_coords)
            
        except Exception as e:
            print(f"経路最適化中にエラーが発生: {e}")
            # エラー時は直線補間を使用
            optimized_coords.append(current_stop["coordinates"])
            
            # 停留所間の中間点を補完（直線補間）
            num_points = 5
            for j in range(1, num_points):
                ratio = j / num_points
                interpolated_lon = current_stop["coordinates"][0] + \
                    (next_stop["coordinates"][0] - current_stop["coordinates"][0]) * ratio
                interpolated_lat = current_stop["coordinates"][1] + \
                    (next_stop["coordinates"][1] - current_stop["coordinates"][1]) * ratio
                optimized_coords.append([interpolated_lon, interpolated_lat])
            
            optimized_coords.append(next_stop["coordinates"])
    
    return optimized_coords

def validate_coordinates(coordinates: List[List[float]]) -> bool:
    """座標データのバリデーション"""
    if not coordinates:
        return False
    
    for coord in coordinates:
        if len(coord) != 2:
            return False
        lon, lat = coord
        # 緯度・経度の基本的な範囲チェック
        if not (-180 <= lon <= 180) or not (-90 <= lat <= 90):
            return False
    return True

In [6]:
import folium
import json

def visualize_routes(geojson_path: str, output_path: str):
    """経路をfoliumで可視化する"""
    
    # 札幌の中心座標
    sapporo_center = [43.0618, 141.3545]
    
    # 地図の初期化
    m = folium.Map(
        location=sapporo_center,
        zoom_start=12,
        tiles='cartodbpositron'
    )
    
    # GeoJSONファイルの読み込み
    with open(geojson_path, 'r', encoding='utf-8') as f:
        route_data = json.load(f)
    
    # 各経路を地図に追加
    for feature in route_data['features']:
        route_id = feature['properties']['route_id']
        route_name = feature['properties']['route_name']
        color = feature['properties'].get('color', '#3388ff')
        
        # 経路の描画
        coordinates = feature['geometry']['coordinates']
        locations = [[coord[1], coord[0]] for coord in coordinates]
        
        folium.PolyLine(
            locations=locations,
            weight=3,
            color=color,
            popup=f'路線ID: {route_id}<br>路線名: {route_name}'
        ).add_to(m)
        
        # 停留所の追加
        for stop in feature['properties']['stops']:
            stop_coords = stop['coordinates']
            folium.CircleMarker(
                location=[stop_coords[1], stop_coords[0]],
                radius=5,
                color=color,
                fill=True,
                popup=f"停留所: {stop['stop_name']}"
            ).add_to(m)
    
    # 凡例の追加
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; right: 50px; width: 150px; height: 90px; 
                border:2px solid grey; z-index:9999; background-color:white;
                opacity:0.8;
                padding: 10px;
                font-size:14px;">
        <p><b>凡例</b></p>
        <p>● 停留所<br>
        ― 路線</p>
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # 地図の保存
    m.save(output_path)
    
    return m

# 経路の可視化
visualize_routes(
    '/app/data/interim/optimized_routes.geojson',
    '/app/data/interim/routes_map.html'
)

print("地図を保存しました: /app/data/interim/routes_map.html")

地図を保存しました: /app/data/interim/routes_map.html


In [7]:
import json
import math

def filter_routes_by_stops(input_path: str, output_path: str, top_percent: float = 0.1):
    """停留所数が多い順に指定された割合の路線を抽出する"""
    
    # GeoJSONファイルの読み込み
    with open(input_path, 'r', encoding='utf-8') as f:
        route_data = json.load(f)
    
    # 各路線の停留所数を計算
    route_stops_count = []
    for feature in route_data['features']:
        route_id = feature['properties']['route_id']
        stops_count = len(feature['properties']['stops'])
        route_stops_count.append({
            'feature': feature,
            'stops_count': stops_count
        })
    
    # 停留所数で降順ソート
    route_stops_count.sort(key=lambda x: x['stops_count'], reverse=True)
    
    # 上位10%の路線数を計算
    total_routes = len(route_stops_count)
    top_routes_count = math.ceil(total_routes * top_percent)
    
    # 上位の路線を抽出
    top_features = [item['feature'] for item in route_stops_count[:top_routes_count]]
    
    # 新しいGeoJSONを作成
    filtered_geojson = {
        "type": "FeatureCollection",
        "features": top_features
    }
    
    # 結果を保存
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_geojson, f, ensure_ascii=False, indent=2)
    
    print(f"全路線数: {total_routes}")
    print(f"抽出路線数: {top_routes_count}")
    print(f"停留所数上位{top_percent*100}%の路線を保存しました: {output_path}")
    
    # 抽出された路線の詳細を表示
    print("\n抽出された路線:")
    for feature in top_features:
        route_id = feature['properties']['route_id']
        route_name = feature['properties']['route_name']
        stops_count = len(feature['properties']['stops'])
        print(f"路線ID: {route_id}, 路線名: {route_name}, 停留所数: {stops_count}")

# 実行
filter_routes_by_stops(
    '/app/data/interim/optimized_routes.geojson',
    '/app/data/interim/optimized_routes_top10percent.geojson',
    0.1
)

全路線数: 33
抽出路線数: 4
停留所数上位10.0%の路線を保存しました: /app/data/interim/optimized_routes_top10percent.geojson

抽出された路線:
路線ID: R010800823, 路線名: 快速７Ｈ札幌駅～豊平峡温泉, 停留所数: 103
路線ID: R010800824, 路線名: 快速７Ｊ札幌駅～定山渓車庫前, 停留所数: 98
路線ID: R010800826, 路線名: 快速８Ｊ札幌駅～定山渓車庫前, 停留所数: 98
路線ID: R010800818, 路線名: １２Ｊ真駒内駅～定山渓車庫前, 停留所数: 76


In [13]:
def calculate_stop_distances(stops: List[dict]) -> List[float]:
    """停留所間の距離を計算する"""
    distances = []
    for i in range(len(stops) - 1):
        current_stop = stops[i]['coordinates']
        next_stop = stops[i + 1]['coordinates']
        
        distance = haversine_distance(
            current_stop[0], current_stop[1],
            next_stop[0], next_stop[1]
        )
        distances.append(distance)
    return distances

def has_outlier_distances(stops: List[dict], threshold_km: float = 5.0) -> bool:
    """停留所間の距離に外れ値があるかチェックする（往復路線を考慮）"""
    distances = calculate_stop_distances(stops)
    
    if not distances:
        return False
    
    # 停留所リストを半分に分割して往路と復路を分析
    half_length = len(stops) // 2
    outbound_stops = stops[:half_length]
    inbound_stops = stops[half_length:]
    
    # 往路の距離を計算
    outbound_distances = calculate_stop_distances(outbound_stops)
    # 復路の距離を計算（存在する場合）
    inbound_distances = calculate_stop_distances(inbound_stops) if inbound_stops else []
    
    # 往路のチェック
    if outbound_distances:
        q1 = np.percentile(outbound_distances, 25)
        q3 = np.percentile(outbound_distances, 75)
        iqr = q3 - q1
        upper_bound = q3 + 1.5 * iqr
        if any(d > threshold_km or d > upper_bound for d in outbound_distances):
            return True
    
    # 復路のチェック
    if inbound_distances:
        q1 = np.percentile(inbound_distances, 25)
        q3 = np.percentile(inbound_distances, 75)
        iqr = q3 - q1
        upper_bound = q3 + 1.5 * iqr
        if any(d > threshold_km or d > upper_bound for d in inbound_distances):
            return True
    
    return False

def filter_routes_by_stops(input_path: str, output_path: str, top_percent: float = 0.1):
    """停留所数が多い順に指定された割合の路線を抽出し、外れ値となる距離を持つ路線を除外する"""
    
    with open(input_path, 'r', encoding='utf-8') as f:
        route_data = json.load(f)
    
    # 各路線の停留所数を計算し、外れ値チェック
    route_stops_count = []
    for feature in route_data['features']:
        route_id = feature['properties']['route_id']
        stops = feature['properties']['stops']
        
        # 外れ値となる距離を持つ路線をスキップ
        if has_outlier_distances(stops):
            print(f"路線 {route_id} は停留所間の距離に外れ値があるためスキップします")
            continue
            
        route_stops_count.append({
            'feature': feature,
            'stops_count': len(stops)
        })
    
    # 停留所数で降順ソート
    route_stops_count.sort(key=lambda x: x['stops_count'], reverse=True)
    
    # 上位10%の路線数を計算
    total_routes = len(route_stops_count)
    top_routes_count = math.ceil(total_routes * top_percent)
    
    # 上位の路線を抽出
    top_features = [item['feature'] for item in route_stops_count[:top_routes_count]]
    
    filtered_geojson = {
        "type": "FeatureCollection",
        "features": top_features
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_geojson, f, ensure_ascii=False, indent=2)
    
    print(f"全路線数: {total_routes}")
    print(f"抽出路線数: {top_routes_count}")
    print(f"停留所数上位{top_percent*100}%の路線を保存しました: {output_path}")

In [15]:
def split_route_stops(stops: List[dict]) -> Tuple[List[dict], List[dict]]:
    """停留所リストを往路と復路に分割する"""
    half_length = len(stops) // 2
    outbound_stops = stops[:half_length]
    return outbound_stops

def filter_routes_by_stops(input_path: str, output_path: str, top_percent: float = 0.1):
    """停留所数が多い順に指定された割合の路線を抽出し、往路のみを保存する"""
    
    with open(input_path, 'r', encoding='utf-8') as f:
        route_data = json.load(f)
    
    route_stops_count = []
    for feature in route_data['features']:
        route_id = feature['properties']['route_id']
        stops = feature['properties']['stops']
        
        # 往路の停留所のみを抽出
        outbound_stops = split_route_stops(stops)
        
        # 外れ値チェック
        if has_outlier_distances(outbound_stops):
            print(f"路線 {route_id} は停留所間の距離に外れ値があるためスキップします")
            continue
        
        # featureを更新（往路のみ）
        feature['properties']['stops'] = outbound_stops
        # 座標も往路のみに更新
        feature['geometry']['coordinates'] = optimize_route_coordinates(outbound_stops, route_id)
            
        route_stops_count.append({
            'feature': feature,
            'stops_count': len(outbound_stops)
        })
    
    # 以下は既存のコードと同じ（ソートと保存）
    route_stops_count.sort(key=lambda x: x['stops_count'], reverse=True)
    total_routes = len(route_stops_count)
    top_routes_count = math.ceil(total_routes * top_percent)
    top_features = [item['feature'] for item in route_stops_count[:top_routes_count]]
    
    filtered_geojson = {
        "type": "FeatureCollection",
        "features": top_features
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(filtered_geojson, f, ensure_ascii=False, indent=2)
    
    print(f"全路線数: {total_routes}")
    print(f"抽出路線数: {top_routes_count}")
    print(f"往路のみの停留所数上位{top_percent*100}%の路線を保存しました: {output_path}")

In [11]:
import numpy as np
from typing import List, Dict, Tuple
from math import radians, sin, cos, sqrt, atan2

def haversine_distance(
    lon1: float,
    lat1: float,
    lon2: float,
    lat2: float
) -> float:
    """2点間の距離をhaversine公式で計算する（単位: km）"""
    R = 6371  # 地球の半径（km）

    # 緯度経度をラジアンに変換
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    
    return distance

def optimize_route_coordinates(
    stops: List[dict],
    route_id: str
) -> List[List[float]]:
    """停留所間の経路を最適化する（直線補間による実装）"""
    optimized_coords = []
    total_distance = 0
    
    for i in range(len(stops) - 1):
        current_stop = stops[i]
        next_stop = stops[i + 1]
        
        # 現在の停留所の座標を追加
        current_coords = current_stop["coordinates"]
        optimized_coords.append(current_coords)
        
        # 停留所間の距離を計算
        distance = haversine_distance(
            current_coords[0], current_coords[1],
            next_stop["coordinates"][0], next_stop["coordinates"][1]
        )
        total_distance += distance
        
        # 距離に応じて補間点の数を調整
        num_points = max(2, min(10, int(distance * 5)))  # 距離に応じて2-10点の間で調整
        
        # 停留所間の中間点を補完
        for j in range(1, num_points):
            ratio = j / num_points
            interpolated_lon = current_coords[0] + (next_stop["coordinates"][0] - current_coords[0]) * ratio
            interpolated_lat = current_coords[1] + (next_stop["coordinates"][1] - current_coords[1]) * ratio
            optimized_coords.append([interpolated_lon, interpolated_lat])
    
    # 最後の停留所の座標を追加
    optimized_coords.append(stops[-1]["coordinates"])
    
    print(f"路線 {route_id} の総距離: {total_distance:.2f}km")
    return optimized_coords

def validate_coordinates(coordinates: List[List[float]]) -> bool:
    """座標データのバリデーション"""
    if not coordinates:
        return False
    
    for coord in coordinates:
        if len(coord) != 2:
            return False
        lon, lat = coord
        # 緯度・経度の基本的な範囲チェック
        if not (-180 <= lon <= 180) or not (-90 <= lat <= 90):
            return False
    return True

In [20]:
filter_routes_by_stops(
    '/app/data/interim/optimized_routes.geojson',
    '/app/data/interim/optimized_routes_filtered.geojson',
    1.0
)

visualize_routes(
    '/app/data/interim/optimized_routes_filtered.geojson',
    '/app/data/interim/routes_map_filtered.html'
)


print("地図を保存しました: /app/data/interim/routes_map_filtered.html")

路線 R010800818 は停留所間の距離に外れ値があるためスキップします
路線 R010800819 は停留所間の距離に外れ値があるためスキップします
路線 R010800005 は停留所間の距離に外れ値があるためスキップします
路線 R010800007 は停留所間の距離に外れ値があるためスキップします
路線 R010800820 の総距離: 6.47km
路線 R010800002 は停留所間の距離に外れ値があるためスキップします
路線 R010800003 は停留所間の距離に外れ値があるためスキップします
路線 R010800821 は停留所間の距離に外れ値があるためスキップします
路線 R010800822 は停留所間の距離に外れ値があるためスキップします
路線 R010800033 は停留所間の距離に外れ値があるためスキップします
路線 R010800823 の総距離: 20.50km
路線 R010800824 の総距離: 19.61km
路線 R010800825 は停留所間の距離に外れ値があるためスキップします
路線 R010800826 は停留所間の距離に外れ値があるためスキップします
路線 R010800009 の総距離: 4.60km
路線 R010800011 は停留所間の距離に外れ値があるためスキップします
路線 R010800013 の総距離: 20.56km
路線 R010800014 の総距離: 2.59km
路線 R010800016 は停留所間の距離に外れ値があるためスキップします
路線 R010800018 は停留所間の距離に外れ値があるためスキップします
路線 R010800020 は停留所間の距離に外れ値があるためスキップします
路線 R010800021 は停留所間の距離に外れ値があるためスキップします
路線 R010800022 は停留所間の距離に外れ値があるためスキップします
路線 R010800023 の総距離: 10.47km
路線 R010800024 は停留所間の距離に外れ値があるためスキップします
路線 R010800025 の総距離: 3.01km
路線 R010800026 は停留所間の距離に外れ値があるためスキップします
路線 R010800027 の総距離: 2.99km
路線 R01080081

In [ ]:
def haversine_distance(lon1: float, lat1: float, lon2: float, lat2: float) -> float:
    """2点間の距離をhaversine公式で計算する（単位: km）"""
    R = 6371  # 地球の半径（km）
    
    # 緯度経度をラジアンに変換
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    
    # haversine公式
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c

def clean_route_linestring(stops: List[dict], max_distance: float = 2.0) -> Tuple[List[dict], bool]:
    """
    停留所リストをクレンジングし、有効な一本のLinestringになるかを判定する
    
    Args:
        stops: 停留所リスト
        max_distance: 許容する最大距離（km）
    
    Returns:
        (クレンジングされた停留所リスト, 有効なLinestringか)
    """
    # 往路のみを抽出（停留所リストの前半）
    half_length = len(stops) // 2
    outbound_stops = stops[:half_length]
    
    if len(outbound_stops) < 2:
        return [], False
    
    # 連続した停留所間の距離をチェック
    valid_segments = []
    current_segment = [outbound_stops[0]]
    
    for i in range(1, len(outbound_stops)):
        current_stop = outbound_stops[i]
        prev_stop = current_segment[-1]
        
        distance = haversine_distance(
            prev_stop['coordinates'][0], prev_stop['coordinates'][1],
            current_stop['coordinates'][0], current_stop['coordinates'][1]
        )
        
        if distance <= max_distance:
            current_segment.append(current_stop)
        else:
            if len(current_segment) > 1:
                valid_segments.append(current_segment)
            current_segment = [current_stop]
    
    if len(current_segment) > 1:
        valid_segments.append(current_segment)
    
    # 有効なセグメントが1つだけ（一本の連続したLinestring）の場合のみ採用
    if len(valid_segments) == 1:
        return valid_segments[0], True
    
    return [], False

def deduplicate_stops(stops: List[dict]) -> List[dict]:
    """
    同名の停留所を中間点に統合する
    
    Args:
        stops: 停留所リスト
    Returns:
        重複を除去した停留所リスト
    """
    # 停留所名でグループ化
    stops_dict = {}
    for stop in stops:
        stop_name = stop['stop_name']
        if stop_name not in stops_dict:
            stops_dict[stop_name] = []
        stops_dict[stop_name].append(stop)
    
    # 重複を統合
    unique_stops = []
    for stop_name, stop_group in stops_dict.items():
        if len(stop_group) == 1:
            # 重複がない場合はそのまま使用
            unique_stops.append(stop_group[0])
        else:
            # 中間点を計算
            avg_lon = sum(s['coordinates'][0] for s in stop_group) / len(stop_group)
            avg_lat = sum(s['coordinates'][1] for s in stop_group) / len(stop_group)
            
            # メトリクスの統合（合計値を使用）
            merged_metrics = {}
            for month in stop_group[0]['metrics'].keys():
                month_metrics = {}
                for metric_key in stop_group[0]['metrics'][month].keys():
                    total = sum(s['metrics'][month][metric_key] for s in stop_group)
                    month_metrics[metric_key] = total
                merged_metrics[month] = month_metrics
            
            # 新しい停留所データを作成
            merged_stop = {
                'stop_id': stop_group[0]['stop_id'],  # 最初の停留所IDを使用
                'stop_name': stop_name,
                'coordinates': [avg_lon, avg_lat],
                'metrics': merged_metrics
            }
            unique_stops.append(merged_stop)
    
    return unique_stops



In [33]:
def filter_and_clean_routes(input_path: str, output_path: str, max_distance: float = 2.0):
    """
    GeoJSONファイルから条件を満たす路線のみを抽出し、停留所の重複を除去する
    """
    with open(input_path, 'r', encoding='utf-8') as f:
        route_data = json.load(f)
    
    cleaned_features = []
    skipped_routes = []
    
    for feature in route_data['features']:
        route_id = feature['properties']['route_id']
        
        # R010800014 を除外
        if route_id == 'R010800014':
            skipped_routes.append(route_id)
            continue
        
        # 停留所の重複を除去
        stops = feature['properties']['stops']
        
        # 往路のみを抽出（停留所リストの前半）
        half_length = len(stops) // 2
        outbound_stops = stops[:half_length]
        
        # 重複停留所の統合
        unique_stops = deduplicate_stops(outbound_stops)
        
        # 距離条件でフィルタリング
        cleaned_stops, is_valid = clean_route_linestring(unique_stops, max_distance)
        
        if not is_valid:
            skipped_routes.append(route_id)
            continue
        
        # 座標リストを更新
        coordinates = [[stop['coordinates'][0], stop['coordinates'][1]] 
                      for stop in cleaned_stops]
        
        # featureを更新
        feature['geometry']['coordinates'] = coordinates
        feature['properties']['stops'] = cleaned_stops
        cleaned_features.append(feature)
    
    # 新しいGeoJSONを作成
    cleaned_geojson = {
        "type": "FeatureCollection",
        "features": cleaned_features
    }
    
    # 結果を保存
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(cleaned_geojson, f, ensure_ascii=False, indent=2)
    
    print(f"処理完了:")
    print(f"- 入力路線数: {len(route_data['features'])}")
    print(f"- 出力路線数: {len(cleaned_features)}")
    print(f"- スキップされた路線数: {len(skipped_routes)}")
    if skipped_routes:
        print("スキップされた路線ID:")
        for route_id in skipped_routes:
            print(f"  - {route_id}")

# 実行
filter_and_clean_routes(
    '/app/data/interim/optimized_routes.geojson',
    '/app/data/interim/cleaned_routes.geojson',
    max_distance=4.0  # 2km以上離れた停留所間を除外
)

# 結果を可視化
visualize_routes(
    '/app/data/interim/cleaned_routes.geojson',
    '/app/data/interim/routes_map_cleaned.html'
)

処理完了:
- 入力路線数: 33
- 出力路線数: 32
- スキップされた路線数: 1
スキップされた路線ID:
  - R010800014


In [37]:
import json
import random
from datetime import datetime, timedelta

def generate_random_metrics(base_metrics, variation=0.1):
    """
    基準となるメトリクスから±10%の範囲でランダムな値を生成
    """
    new_metrics = {}
    for key, value in base_metrics.items():
        if isinstance(value, dict):
            new_metrics[key] = generate_random_metrics(value)
        elif isinstance(value, (int, float)):
            variation_amount = value * variation
            new_value = value + random.uniform(-variation_amount, variation_amount)
            # 整数の場合は整数に丸める
            if isinstance(value, int):
                new_value = int(round(new_value))
            new_metrics[key] = new_value
        else:
            new_metrics[key] = value
    return new_metrics

def add_monthly_metrics(input_path, output_path):
    # GeoJSONを読み込む
    print("ファイルを読み込み中...")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # 6ヶ月分の年月を生成（202311から始める）
    base_date = datetime(2023, 11, 1)
    months = [(base_date + timedelta(days=30*i)).strftime("%Y%m") 
             for i in range(6)]
    print(f"生成された月リスト: {months}")
    
    print("メトリクスを更新中...")
    # 各featureに対して処理
    for feature in data['features']:
        base_metrics = feature['properties']['metrics']['202311']
        
        # 路線全体のメトリクス
        for month in months:
            if month != '202311':  # 既存の202311は保持
                feature['properties']['metrics'][month] = generate_random_metrics(base_metrics)
        
        # 各停留所のメトリクス
        for stop in feature['properties']['stops']:
            base_stop_metrics = stop['metrics']['202311']
            for month in months:
                if month != '202311':
                    stop['metrics'][month] = generate_random_metrics(base_stop_metrics)
    
    # 結果を保存
    print(f"結果を保存中: {output_path}")
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print("処理が完了しました")

# 実行
add_monthly_metrics(
    '/app/data/interim/cleaned_routes.geojson',
    '/app/data/interim/cleaned_routes_with_months.geojson'
)

visualize_routes(
    '/app/data/interim/cleaned_routes_with_months.geojson',
    '/app/data/interim/cleaned_routes_with_months.html'
)

ファイルを読み込み中...
生成された月リスト: ['202311', '202312', '202312', '202401', '202402', '202403']
メトリクスを更新中...
結果を保存中: /app/data/interim/cleaned_routes_with_months.geojson
処理が完了しました
